In [1]:
%pip install geopy requests


                                              0.0/125.4 kB ? eta -:--:--
     ---                                      10.2/125.4 kB ? eta -:--:--
     ---                                      10.2/125.4 kB ? eta -:--:--
     ---------                             30.7/125.4 kB 163.8 kB/s eta 0:00:01
     ---------                             30.7/125.4 kB 163.8 kB/s eta 0:00:01
     ---------                             30.7/125.4 kB 163.8 kB/s eta 0:00:01
     ---------                             30.7/125.4 kB 163.8 kB/s eta 0:00:01
     ---------                             30.7/125.4 kB 163.8 kB/s eta 0:00:01
     ---------                             30.7/125.4 kB 163.8 kB/s eta 0:00:01
     ------------                           41.0/125.4 kB 78.6 kB/s eta 0:00:02
     ------------                           41.0/125.4 kB 78.6 kB/s eta 0:00:02
     ------------------                    61.4/125.4 kB 109.2 kB/s eta 0:00:01
     ---------------------                 71.7/125.4 kB 1

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import requests
import time

def find_nearest_hospital_osm(pin_code):
    # Geocode the pin code to get latitude and longitude
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(pin_code, timeout=10)
    
    if not location:
        print("Invalid pin code.")
        return None
    
    lat, lng = location.latitude, location.longitude
    
    # Use Overpass API to find nearby hospitals
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    node["amenity"="hospital"](around:5000,{lat},{lng});
    out;
    """
    
    response = requests.get(overpass_url, params={'data': overpass_query})
    
    if response.status_code != 200:
        print("Error retrieving data from Overpass API.")
        return None
    
    data = response.json()
    if not data['elements']:
        print("No hospitals found nearby.")
        return None
    
    # Extract the nearest hospital details
    nearest_hospital = data['elements'][0]
    hospital_name = nearest_hospital.get('tags', {}).get('name', 'Unnamed Hospital')
    hospital_lat = nearest_hospital['lat']
    hospital_lon = nearest_hospital['lon']
    
    return {
        "name": hospital_name,
        "location": {"latitude": hospital_lat, "longitude": hospital_lon}
    }


# Usage example
pin_code = "673591"
nearest_hospital = find_nearest_hospital_osm(pin_code)

if nearest_hospital:
    print("Nearest Hospital:")
    print("Name:", nearest_hospital['name'])
    print("Location:", nearest_hospital['location'])



Nearest Hospital:
Name: PBM Hospital
Location: {'latitude': 11.6570597, 'longitude': 76.1596653}
